In [ ]:
import numpy as n
import pandas as p
import matplotlib.pyplot as graph
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class NeuralNetwork:
  def __init__(self, data, target, lRate = 0.001):
    self.data = n.array(data)
    self.weightSequence = []
    self.biasSequence = []
    self.activations = []
    self.y = n.array(target)
    self.lr = lRate

  def sigmoid(self, z):
    return 1/(1 + n.exp(-z))

  def layer(self, units):
    size = self.data.shape[1]

    if len(self.weightSequence) != 0:
      size = self.weightSequence[-1].shape[0]

    weightLayer = n.array([n.array(n.random.random(size)/1000) for i in range(units)])
    biasLayer = n.array([0.0 for i in range(units)])

    self.weightSequence.append(weightLayer)
    self.biasSequence.append(biasLayer)

  def forwardProp(self, data):
    self.activations = []
    weights = self.weightSequence
    bias = self.biasSequence
    layerCount = len(weights)

    layerInput = data

    for i in range(layerCount):
      w = weights[i]
      b = bias[i]

      layerOutput = n.matmul(layerInput, w.T) + b
      self.activations.append(layerOutput)
      layerInput = layerOutput

    return layerInput[0] if len(layerInput) == 1 else layerInput

  def backProp(self):
    self.forwardProp(self.data)
    # print("\n",self.activations)
    m = self.data.shape[0]
    y_cap = self.sigmoid(self.activations[-1])
    error = y_cap - self.y
    # print(y_cap, self.y)
    # print("error",error)
    # print("activation",self.activations[-2])

    dj_dw = n.dot(self.activations[-2].T, error)/m
    dj_db = n.sum(error, axis = 0, keepdims=True)/m
    # print(dj_dw, dj_db)


    output_weights = self.weightSequence[-1]
    output_bias = self.biasSequence[-1]

    # print(output_weights, output_bias)

    output_weights -= self.lr*dj_dw.T
    output_bias -= self.lr*dj_db[0]

    for i in range(len(self.weightSequence) - 2, -1, -1):
      # print(i, "\n--------------")

      dj_dyi = error
      prevActivation = self.activations[i-1] if i-1 >= 0 else self.data
      # print(dj_dyi, "\n\n", self.weightSequence[i+1], "\n\n", prevActivation)
      error = n.matmul(dj_dyi, self.weightSequence[i+1])

      dj_dw = n.matmul(error.T, prevActivation)/m
      dj_db = n.sum(error, axis = 0)/m

      self.weightSequence[i] -= self.lr*dj_dw
      self.biasSequence[i] -= self.lr*dj_db

  def trainModel(self, iter = 1000, lr = 0.0001):
      self.lr = lr
      for i in range(iter):
        self.backProp()


In [ ]:
dataSheet = p.read_csv(f"gdrive/My Drive/ML WOC/datasheets/nn_train.csv")
data = n.array(dataSheet)

In [ ]:
x = data[:100, 1:-2]
y = data[:100, -2].reshape(x.shape[0], 1)


nn = NeuralNetwork(x, y)
nn.layer(128)
nn.layer(64)
nn.layer(32)
nn.layer(1)

# print(nn.weightSequence, "\n")
# print(nn.biasSequence, "\n")

nn.trainModel(50000, 0.001)

# print(nn.weightSequence, "\n")
# print(nn.biasSequence, "\n")

# nn.forwardProp([[1, 2, 3], [4, 5, 6]])
# print(nn.sigmoid(nn.activations[-1]))
# print("+++++++++++++++++")
# for i in nn.activations:
#   print("\n", i)
# print("\n", nn.weightSequence)
# print(nn.biasSequence)



In [ ]:
nn.forwardProp(data[:, 1:-2])
a = nn.sigmoid(nn.activations[-1])
prediction = n.select([a <= .5, a>.5], [n.zeros_like(a), n.ones_like(a)])
actual = data[:, -2]
# for i in range(len(nn.sigmoid(nn.activations[-1]))):
#   print(nn.sigmoid(nn.activations[-1][i]), actual[i])
acc = 0
for i in range(len(actual)):
  # print("Actual",actual[i],"Predicted",prediction[i])
  if actual[i] == prediction[i]:
    acc += 1
print(acc/len(actual))

0.5870375


In [ ]:
dataSheet_t = p.read_csv(f"gdrive/My Drive/ML WOC/datasheets/nn_test.csv")
data_t = n.array(dataSheet)

In [40]:
data_test = list(data_t )
data_with_target_test = []
lastColumnIndex = len(data_test[0])

for i in data_test:
  features = []
  prediction = 1 if nn.sigmoid(nn.forwardProp(i[1:-2])) >= 0.5 else 0
  features.append(prediction)
  data_with_target_test.append(features)

data_with_target_test = n.array(data_with_target_test)
df_with_target_test = p.DataFrame(data_with_target_test , columns=["Binary_predictions"])
df_with_target_test.to_csv('nn_test_predictions_binary_labels.csv', index=True)